In [13]:
import os

import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

from osgeo import gdal
from scipy.io import loadmat
from scipy.fft import ifft, fft, fftshift, ifftshift, fftfreq
from scipy.fft import ifft2, fft2
from scipy.constants import pi, c

In [14]:
PRIMARY_HEADER = [
    3,   # Packet Version Number
    1,   # Packet Type
    1,   # Secondary Header Flag
    7,   # Process ID
    4,   # Process Category
    2,   # Sequence Flags
    14,  # Packet Sequence Count 
    16   # Packet Data Length
]

PRIMARY_HEADER_FIELDS = [
    'packet_version_number',
    'packet_type',
    'secondary_header_flag',
    'process_id',
    'process_category',
    'sequence_flags',
    'packet_sequence_count',
    'packet_data_length'
]

SECONDARY_HEADER = [
    32,  # Coarse Time
    16,  # Fine Time
    32,  # Sync Marker
    32,  # Data Take ID
    8,   # ECC Number
    1,   # N/A
    3,   # Test Mode
    4,   # RX Channel ID
    32,  # Instrument Configuration ID
    8,   # Sub-Commutative Ancillary Data Word Index
    16,  # Sub-Commutative Ancillary Data Word
    32,  # Counter Service
    32,  # PRI Count
    1,   # Error Flag
    2,   # N/A
    5,   # BAQ Mode
    8,   # BAQ Block Length
    8,   # N/A
    8,   # Range Decimation
    8,   # RX Gain
    16,  # TX Ramp Rate
    16,  # Pulse Start Frequency
    24,  # Pulse Length
    3,   # N/A
    5,   # Rank
    24,  # PRI
    24,  # SWST
    24,  # SWL
    24,  # SAS SSB Message *** TODO: See Page 43, 44
    24,  # SES SSB Message *** TODO: See Page 50
    16,  # Number of Quadratures
    8,   # N/A
]

SECONDARY_HEADER_FIELDS = [
    'coarse_time',
    'fine_time',
    'sync_marker',
    'data_take_id',
    'ecc_number',
    'na_1',
    'test_mode',
    'rx_channel_id',
    'instrument_configuration_id',
    'sc_data_word_index',
    'sc_data_word',
    'counter_service',
    'pri_count',
    'error_flag',
    'na_2',
    'baq_mode',
    'baq_block_length',
    'na_3',
    'range_decimation',
    'rx_gain',
    'tx_ramp_rate',
    'pulse_start_frequency',
    'pulse_length',
    'na_4',
    'rank',
    'pri',
    'swst',
    'swl',
    'sas_ssb_message',
    'ses_ssb_message',
    'num_quadratures',
    'na_5',
]

SUB_COMMUTATIVE_DATA_SERVICE = [
    16,      # Dummy Data
    64,      # X-Axis ECEF Position
    64,      # Y-Axis ECEF Position
    64,      # Z-Axis ECEF Position
    32,      # X-velocity ECEF
    32,      # Y-velocity ECEF
    32,      # Z-velocity ECEF
    16,      # POD Data Stamp 1
    16,      # Pod Data Stamp 2
    16,      # Pod Data Stamp 3
    16,      # Pod Data Stamp 4
    32,      # Q0 Attitude Quaternion
    32,      # Q1 Attitude Quaternion
    32,      # Q2 Attitude Quaternion
    32,      # Q3 Attitude Quaternion
    32,      # OmegaX Angular Rate
    32,      # OmegaY Angular Rate
    32,      # OmegaZ Angular Rate
    16,      # Data Time Stamp 1
    16,      # Data Time Stamp 2
    16,      # Data Time Stamp 3
    16,      # Data Time Stamp 4
    16,      # Pointing Status
    16,      # Temperature Update Status
    8, 8, 8, # Tile 1 EFE H, V Temperature and Activate TA Temperature
    8, 8, 8, # Tile 2
    8, 8, 8, # Tile 3
    8, 8, 8, # Tile 4
    8, 8, 8, # Tile 5
    8, 8, 8, # Tile 6
    8, 8, 8, # Tile 7
    8, 8, 8, # Tile 8
    8, 8, 8, # Tile 9
    8, 8, 8, # Tile 10
    8, 8, 8, # Tile 11
    8, 8, 8, # Tile 12
    8, 8, 8, # Tile 13
    8, 8, 8, # Tile 14
    9,       # N/A
    7,       # TGU Temperature
]

In [52]:
def create_bit_string(bytes_string: str):
    bit_string = ''
    for byte in bytes_string:
        bit_string += '{:08b}'.format(byte)
    return bit_string


read_and_pop = lambda bit_string, bit_length: (bit_string[0:bit_length], bit_string[bit_length:])


def bit_len_map(brc: int):
    k = 1
    match brc:
        case 0:
            k = 4
        case 1:
            k = 5
        case 2:
            k = 7
        case 3:
            k = 10
        case 4:
            k = 16
    return k


def huffman(bit_string: str, brc: int):
    k = bit_len_map(brc)
    bits, bit_string = read_and_pop(bit_string, k)
    m_code = 0
    bit_len = 0
    for bit in bits:
        bit_len += 1
        if bit == '0':
            break
        else:
            m_code += 1
    return m_code, bit_string, bit_len


def get_s_codes(
    bit_string: str,
    brc: int,
) -> ([int], [int], str):
    signs = []
    m_codes = []
    s_code_count = 0
    while s_code_count < 128:
        sign, bit_string = read_and_pop(bit_string, 1)
        m_code, bit_string, bit_len = huffman(bit_string, brc)
        signs.append(sign)
        m_codes.append(m_code)
        s_code_count += 1
    return  signs, m_codes, bit_string


def get_s_codes_qe(
    bit_string: str,
    brc: int,
) -> ([int], [int], [int], str):
    signs = []
    m_codes = []       
    threshold, bit_string = read_and_pop(bit_string, 8)
    s_code_count = 0
    while s_code_count < 128:
        sign, bit_string = read_and_pop(bit_string, 1)
        m_code, bit_string, bit_len = huffman(bit_string, brc)
        signs.append(sign)
        m_codes.append(m_code)
        s_code_count += 1
    return signs, m_codes, [threshold], bit_string

In [53]:
s = '010010101010100111'
m, bs, bl = huffman(s, 0)
print(f"M_CODE: {m}")
print(f"BIT_STRING: {bs}")
print(f"BIT_LENGTH: {bl}")

M_CODE: 0
BIT_STRING: 10101010100111
BIT_LENGTH: 1


In [54]:
class Packet:
    def __init__(
        self,
        primary_header,
        secondary_header,
        user_data_field
    ):
        self.primary_header   = primary_header
        self.secondary_header = secondary_header
        self.raw_user_data    = user_data_field

        self.version_number        = int(primary_header['packet_version_number'], 2)
        self.type                  = int(primary_header['packet_type'], 2)
        self.secondary_header_flag = int(primary_header['secondary_header_flag'], 2)
        self.process_id            = int(primary_header['process_id'], 2)
        self.process_category      = int(primary_header['process_category'], 2)
        self.sequence_flags        = int(primary_header['sequence_flags'])
        self.sequence_count        = int(primary_header['packet_sequence_count'], 2)
        self.packet_data_length    = int(primary_header['packet_data_length'], 2)
        self.user_data_length      = (self.packet_data_length + 1) - 62

        self.num_quads   = int(self.secondary_header['num_quadratures'], 2)
        self.num_samples = 2 * self.num_quads

        self.test_mode      = int(self.secondary_header['test_mode'], 2)
        self.baq_mode       = int(self.secondary_header['baq_mode'], 2)
        self.block_length   = int(self.secondary_header['baq_block_length'], 2)
        self.num_baq_blocks = int(np.ceil(2 * self.num_quads / 256))

        self._set_data_format()
        self._decode_user_data_field()

    
    def _set_num_words_type_c(self):
        num_words_ie = int(np.ceil(self.block_length * self.num_quads / 16))
        num_words_qe = int(np.ceil((self.block_length * self.num_quads + 8 * self.num_baq_blocks) / 16))
        self.num_words = (num_words_ie, num_words_qe)

    
    def _decode_type_c_data(self):    
        self._set_num_words_type_c()
        s_code_width = 0
        bit_string = create_bit_string(self.raw_user_data)
        block_width = (128 * self.block_length - 1) * self.num_baq_blocks

        self.ie_signs = []
        self.ie_m_codes = []
        self.io_signs = []
        self.io_m_codes = []
        self.qe_signs = []
        self.qe_m_codes = []
        self.qo_signs = []
        self.qo_m_codes = []
        self.thresholds = []
        for i in range(self.num_baq_blocks):
            brc, bit_string = read_and_pop(bit_string, 3)
            if brc != '':
                brc = int(brc, 2)
                bit_len = bit_len_map(brc)
                thresholds = []
                if i <= self.num_baq_blocks - 1:
                    signs, m_codes, bit_string = get_s_codes(bit_string, brc)
                    self.ie_signs.append(signs)
                    self.ie_m_codes.append(m_codes)
                    signs, m_codes, bit_string = get_s_codes(bit_string, brc)
                    self.io_signs.append(signs)
                    self.io_m_codes.append(m_codes)
                    signs, m_codes, threshold, bit_string = get_s_codes_qe(bit_string, brc)
                    self.qe_signs.append(signs)
                    self.qe_m_codes.append(m_codes)
                    self.thresholds.append(threshold)
                    signs, m_codes, bit_string = get_s_codes(bit_string, brc)
                    self.qo_signs.append(signs)
                    self.qo_m_codes.append(m_codes)


    def _decode_user_data_field(self):
        if self.data_format == 'A':
            pass
        elif self.data_format == 'B':
            pass
        elif self.data_format == 'C':
            pass
        elif self.data_format == 'D':
            self._decode_type_c_data()



    def _set_data_format(self):
        self.test_mode = int(self.secondary_header['test_mode'], 2)
        self.baq_mode  = int(self.secondary_header['baq_mode'], 2)
        if self.baq_mode == 0:
            if self.test_mode % 3 == 0:
                self.data_format = 'A'
            else:
                self.data_format = 'B'
        elif self.baq_mode in [3, 4, 5]:
            self.data_format = 'C'
        elif self.baq_mode in [12, 13, 14]:
            self.data_format = 'D'
        else:
            raise ValueError(f'No Data Format with BAQ Mode: {self.baq_mode} and Test Mode: {self.test_mode}')

In [76]:
def get_header_dict(header_bytes, header_bit_lengths, header_field_names):
    read_and_pop = lambda bit_string, bit_length: (bit_string[0:bit_length], bit_string[bit_length:])
    bit_string = ''
    for byte in header_bytes:
        bit_string += '{:08b}'.format(byte)

    header_value_array = []
    for bit_len in header_bit_lengths:
        header_field_value, bit_string = read_and_pop(bit_string, bit_len)
        header_value_array.append(header_field_value) 

    index = 0
    header_dict = {}
    for field in header_field_names:
        if index >= len(header_field_names) or index >= len(header_value_array):
            raise ValueError('The number of field names and header values does not match.')
        header_dict[field] = header_value_array[index]
        index += 1

    return header_dict


def packet_generator(raw_data):
    secondary_header_length = 62
    while raw_data:
        primary_header_bytes = raw_data.read(6)
        primary_header = get_header_dict(primary_header_bytes, PRIMARY_HEADER, PRIMARY_HEADER_FIELDS)
        secondary_header_bytes = raw_data.read(62)
        secondary_header = get_header_dict(secondary_header_bytes, SECONDARY_HEADER, SECONDARY_HEADER_FIELDS)
        packet_data_length = primary_header['packet_data_length']
        user_data = None
        if packet_data_length:
            packet_data_length = int(packet_data_length, 2)
            user_data_length = (packet_data_length + 1) - secondary_header_length
            if user_data_length > 0:
                user_data = raw_data.read(user_data_length)
        yield Packet(
            primary_header = primary_header,
            secondary_header = secondary_header,
            user_data_field = user_data
        )


# def pulse_constructor(raw_data):
#     pg = packet_generator(raw_data)
#     initial_packet = next(pg)

In [56]:
l0_data_file = 'sar_data/S1A_IW_RAW__0SDV_20240806T135224_20240806T135256_055093_06B68A_AE41.SAFE/s1a-iw-raw-s-vv-20240806t135224-20240806t135256-055093-06b68a.dat'

In [57]:
raw_data_file = open(l0_data_file, 'rb')

In [58]:
PacketGenerator = packet_generator(raw_data_file)

In [59]:
packet = next(PacketGenerator)

In [60]:
packet.primary_header

{'packet_version_number': '000',
 'packet_type': '0',
 'secondary_header_flag': '1',
 'process_id': '1000001',
 'process_category': '1100',
 'sequence_flags': '11',
 'packet_sequence_count': '10101110010101',
 'packet_data_length': '0100100010111101'}

In [61]:
packet.secondary_header

{'coarse_time': '01010011110111001110110100101010',
 'fine_time': '0001111001001101',
 'sync_marker': '00110101001011101111100001010011',
 'data_take_id': '00001101011011010001010111000000',
 'ecc_number': '00001000',
 'na_1': '0',
 'test_mode': '000',
 'rx_channel_id': '0000',
 'instrument_configuration_id': '00000000000000000000000000000111',
 'sc_data_word_index': '00011011',
 'sc_data_word': '1011111011000011',
 'counter_service': '00000000000000111010101110010101',
 'pri_count': '00000000000000111011011010111000',
 'error_flag': '0',
 'na_2': '00',
 'baq_mode': '01100',
 'baq_block_length': '00011111',
 'na_3': '00000000',
 'range_decimation': '00001000',
 'rx_gain': '00001000',
 'tx_ramp_rate': '1000011001000101',
 'pulse_start_frequency': '0011000000101111',
 'pulse_length': '000000000000011110101111',
 'na_4': '000',
 'rank': '01001',
 'pri': '000000000101010101100011',
 'swst': '000000000000111001100001',
 'swl': '000000000011011010011011',
 'sas_ssb_message': '011111000110000

In [62]:
next_packet = next(PacketGenerator)

In [63]:
next_packet.primary_header

{'packet_version_number': '000',
 'packet_type': '0',
 'secondary_header_flag': '1',
 'process_id': '1000001',
 'process_category': '1100',
 'sequence_flags': '11',
 'packet_sequence_count': '10101110010110',
 'packet_data_length': '0100100010011001'}

In [64]:
next_packet.secondary_header

{'coarse_time': '01010011110111001110110100101010',
 'fine_time': '0001111001110011',
 'sync_marker': '00110101001011101111100001010011',
 'data_take_id': '00001101011011010001010111000000',
 'ecc_number': '00001000',
 'na_1': '0',
 'test_mode': '000',
 'rx_channel_id': '0000',
 'instrument_configuration_id': '00000000000000000000000000000111',
 'sc_data_word_index': '00011100',
 'sc_data_word': '1111000100001001',
 'counter_service': '00000000000000111010101110010110',
 'pri_count': '00000000000000111011011010111001',
 'error_flag': '0',
 'na_2': '00',
 'baq_mode': '01100',
 'baq_block_length': '00011111',
 'na_3': '00000000',
 'range_decimation': '00001000',
 'rx_gain': '00001000',
 'tx_ramp_rate': '1000011001000101',
 'pulse_start_frequency': '0011000000101111',
 'pulse_length': '000000000000011110101111',
 'na_4': '000',
 'rank': '01001',
 'pri': '000000000101010101100011',
 'swst': '000000000000111001100001',
 'swl': '000000000011011010011011',
 'sas_ssb_message': '011111000110000

In [65]:
packet.secondary_header == next_packet.secondary_header

False

In [66]:
packet.num_samples == next_packet.num_samples

True

In [67]:
packet.data_format == next_packet.data_format == 'D'

True

In [68]:
packet.user_data_length

18560

In [69]:
next_packet.sequence_count

11158

In [70]:
next_packet.num_baq_blocks

94

In [71]:
len(packet.ie_m_codes)

55

In [72]:
ie_len = 0
io_len = 0
qe_len = 0
qo_len = 0
for arr in packet.ie_m_codes:
    ie_len += len(arr)
for arr in packet.io_m_codes:
    io_len += len(arr)
for arr in packet.qe_m_codes:
    qe_len += len(arr)
for arr in packet.qo_m_codes:
    qo_len += len(arr)
print(f"IE M_CODES: {ie_len}")
print(f"IO M_CODES: {io_len}")
print(f"QE M_CODES: {qe_len}")
print(f"QO M_CODES: {qo_len}")

IE M_CODES: 7040
IO M_CODES: 7040
QE M_CODES: 7040
QO M_CODES: 7040


In [73]:
packet.io_m_codes

[[0,
  1,
  1,
  0,
  1,
  0,
  0,
  1,
  0,
  2,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  2,
  0,
  1,
  0,
  0,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  2,
  0,
  0,
  2,
  0,
  1,
  3,
  0,
  1,
  0,
  1,
  2,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  3,
  0,
  2,
  0,
  2,
  0,
  1,
  0,
  1,
  1,
  0,
  3,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  1,
  3,
  0,
  0,
  2,
  3,
  0,
  0,
  2,
  1,
  1,
  0,
  0,
  0,
  0,
  1,
  4,
  0,
  0,
  1,
  1,
  0,
  1,
  2,
  0,
  0,
  1,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  3,
  2,
  0,
  1,
  0,
  1,
  1,
  0,
  0,
  1,
  1,
  3,
  0],
 [2,
  0,
  0,
  1,
  1,
  1,
  0,
  1,
  2,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  1,
  2,
  2,
  1,
  0,
  1,
  2,
  0,
  2,
  0,
  0,
  1,
  1,
  5,
  0,
  1,
  1,
  1,
  3,
  0,
  0,
  3,
  0,
  2,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  2,
  0,
  0,
  4,
  0,
  0,
  2,
  3,
  2,
  2,
  0,
  0,
  0,
  2,
  1,
  0,
  1,
  0,
  2,

In [74]:
packet.num_samples

23876